In [31]:
# Initial imports
import os
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

In [32]:
# Load .env environment variables
env_path = Path("./alpaca_keys.env")
load_dotenv(dotenv_path=env_path)

Python-dotenv could not parse statement starting at line 1


True

In [33]:

API_KEY = os.getenv("ALPACA_API_KEY")
API_SECRET = os.getenv("ALPACA_SECRET_KEY")
ALPACA_API_BASE_URL = "https://paper-api.alpaca.markets"

In [34]:
# Create a connection to the API 
alpaca = tradeapi.REST(
    API_KEY,
    API_SECRET,
    ALPACA_API_BASE_URL,
    api_version="v2")

In [35]:
# Set the tickers
eth_ticker = ["ETHUSD"]
btc_ticker = ["BTCUSD"]


In [36]:
# Set the parameter tz to "America/New_York", 

start_date = pd.Timestamp("2016-02-08", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-02-08", tz="America/New_York").isoformat()

In [37]:
# Set timeframe to one day (1D) for the Alpaca API
timeframe = "1Day"

In [38]:
# Use the Alpaca get_crypto_bars function to gather the price information for ETHEREUM

eth_data_df = alpaca.get_crypto_bars(
    eth_ticker,
    timeframe,
    start = start_date,
    end = end_date
).df


# Review the resulting `portfolio_prices_df` DataFrame. 
display(eth_data_df.head())


,exchange,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,,
2016-05-17 05:00:00+00:00,CBSE,12.50,14.93,12.50,14.00,23.517331,15,13.627887,ETHUSD
2016-05-18 05:00:00+00:00,CBSE,13.60,13.60,13.03,13.18,459.004496,65,13.101801,ETHUSD
2016-05-19 05:00:00+00:00,CBSE,14.82,14.82,14.82,14.82,16.000000,2,14.820000,ETHUSD
2016-05-20 05:00:00+00:00,CBSE,13.72,14.17,13.71,14.17,238.166617,17,13.776081,ETHUSD
2016-05-23 05:00:00+00:00,CBSE,13.68,13.74,13.68,13.74,1.000000,3,13.720367,ETHUSD


In [42]:
# Calculate the daily returns using the closing prices and the pct_change function
eth_data_df["actual returns"] = eth_data_df["close"].pct_change()

# Drop all NaN values from the DataFrame
eth_data_df = eth_data_df.dropna()

In [43]:
eth_prices_df = eth_data_df[['symbol', 'open', 'high', 'low', 'close', 'volume', 'actual returns']]
display(eth_prices_df.head(10))
display(eth_prices_df.tail(10))

,symbol,open,high,low,close,volume,actual returns
timestamp,,,,,,,
2016-05-18 05:00:00+00:00,ETHUSD,13.60,13.60,13.03,13.18,459.004496,-0.058571
2016-05-19 05:00:00+00:00,ETHUSD,14.82,14.82,14.82,14.82,16.000000,0.124431
2016-05-20 05:00:00+00:00,ETHUSD,13.72,14.17,13.71,14.17,238.166617,-0.043860
2016-05-23 05:00:00+00:00,ETHUSD,13.68,13.74,13.68,13.74,1.000000,-0.030346
2016-05-24 05:00:00+00:00,ETHUSD,13.00,13.18,12.61,12.94,2812.179184,-0.058224
2016-05-25 05:00:00+00:00,ETHUSD,12.61,12.95,12.61,12.95,510.572751,0.000773
2016-05-26 05:00:00+00:00,ETHUSD,12.47,12.47,11.25,11.49,6099.312961,-0.112741
2016-05-27 05:00:00+00:00,ETHUSD,11.48,11.70,10.25,10.98,13257.989737,-0.044386
2016-05-28 05:00:00+00:00,ETHUSD,11.19,12.44,10.73,12.02,8612.425253,0.094718


,symbol,open,high,low,close,volume,actual returns
timestamp,,,,,,,
2022-02-04 06:00:00+00:00,ETHUSD,2773.50,3040.00,2765.20,3023.50,22296.968000,-0.001826
2022-02-05 06:00:00+00:00,ETHUSD,3027.43,3063.48,2962.12,3022.17,123306.749331,-0.000440
2022-02-05 06:00:00+00:00,ETHUSD,3030.92,3060.27,2963.90,3017.18,1903.139901,-0.001651
2022-02-05 06:00:00+00:00,ETHUSD,3027.10,3062.30,2963.00,3021.60,11510.016000,0.001465
2022-02-06 06:00:00+00:00,ETHUSD,3021.93,3112.82,2950.26,3094.99,119161.605220,0.024288
2022-02-06 06:00:00+00:00,ETHUSD,3010.18,3101.16,2967.57,3095.09,2242.796106,0.000032
2022-02-06 06:00:00+00:00,ETHUSD,3020.00,3112.00,2954.00,3092.70,12299.525000,-0.000772
2022-02-07 06:00:00+00:00,ETHUSD,3095.00,3199.08,3051.05,3187.58,220032.905103,0.030679
2022-02-07 06:00:00+00:00,ETHUSD,3098.79,3197.50,3062.28,3191.60,3625.869325,0.001261


In [16]:
# Use the Alpaca get_crypto_bars function to gather the price information for BITCOIN
btc_data_df = alpaca.get_crypto_bars(
    btc_ticker,
    timeframe,
    start = start_date,
    end = end_date
).df

display(btc_data_df.head())

,exchange,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,,
2016-02-08 06:00:00+00:00,CBSE,371.02,372.13,370.53,371.35,1895.115345,3251,371.511186,BTCUSD
2016-02-09 06:00:00+00:00,CBSE,372.76,374.95,371.87,374.57,2394.781486,6108,373.284012,BTCUSD
2016-02-10 06:00:00+00:00,CBSE,378.49,379.50,374.80,374.84,3417.150130,9538,377.184969,BTCUSD
2016-02-11 06:00:00+00:00,CBSE,375.01,378.27,361.10,378.23,6923.029859,19822,374.110998,BTCUSD
2016-02-12 06:00:00+00:00,CBSE,382.05,396.00,381.99,390.21,2787.431249,6509,388.104739,BTCUSD


In [44]:
# Calculate the daily returns using the closing prices and the pct_change function
btc_data_df["actual returns"] = btc_data_df["close"].pct_change()

# Drop all NaN values from the DataFrame
btc_data_df = btc_data_df.dropna()

In [47]:
btc_prices_df = btc_data_df[['symbol', 'open', 'high', 'low', 'close', 'volume', "actual returns"]]

display(btc_prices_df.head(10))
display(btc_prices_df.tail(10))

,symbol,open,high,low,close,volume,actual returns
timestamp,,,,,,,
2016-02-09 06:00:00+00:00,BTCUSD,372.76,374.95,371.87,374.57,2394.781486,0.008671
2016-02-10 06:00:00+00:00,BTCUSD,378.49,379.50,374.80,374.84,3417.150130,0.000721
2016-02-11 06:00:00+00:00,BTCUSD,375.01,378.27,361.10,378.23,6923.029859,0.009044
2016-02-12 06:00:00+00:00,BTCUSD,382.05,396.00,381.99,390.21,2787.431249,0.031674
2016-02-13 06:00:00+00:00,BTCUSD,391.00,398.82,391.00,397.05,2765.507131,0.017529
2016-02-14 06:00:00+00:00,BTCUSD,406.62,409.09,400.00,405.08,2087.536940,0.020224
2016-02-15 06:00:00+00:00,BTCUSD,405.09,406.87,394.03,398.95,4480.165423,-0.015133
2016-02-16 06:00:00+00:00,BTCUSD,399.73,402.66,399.68,402.15,506.573415,0.008021
2016-02-17 06:00:00+00:00,BTCUSD,415.18,419.98,415.18,419.13,1367.450154,0.042223


,symbol,open,high,low,close,volume,actual returns
timestamp,,,,,,,
2022-02-04 06:00:00+00:00,BTCUSD,37573.00,41940.00,37332.00,41533.00,2727.720100,0.003016
2022-02-05 06:00:00+00:00,BTCUSD,41559.01,41934.38,40955.40,41610.64,8115.040087,0.001869
2022-02-05 06:00:00+00:00,BTCUSD,41479.00,41914.00,41015.00,41499.70,73.036337,-0.002666
2022-02-05 06:00:00+00:00,BTCUSD,41561.00,41921.00,40991.00,41609.00,783.941400,0.002634
2022-02-06 06:00:00+00:00,BTCUSD,41610.65,43036.87,41135.39,42835.99,11326.596609,0.029489
2022-02-06 06:00:00+00:00,BTCUSD,41683.00,43014.00,41189.00,42770.60,371.202088,-0.001527
2022-02-06 06:00:00+00:00,BTCUSD,41579.00,43031.00,41135.00,42816.00,1301.866700,0.001061
2022-02-07 06:00:00+00:00,BTCUSD,42835.26,45035.00,42258.61,44916.93,17605.802688,0.049069
2022-02-07 06:00:00+00:00,BTCUSD,42779.70,44997.40,42290.10,44983.70,660.084832,0.001487
